In [1]:
import numpy as np
import pandas as pd
from scipy.special import digamma
from scipy.spatial import KDTree
import time
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler

#### Importing the dataset

In [2]:
!pip install ucimlrepo

In [3]:
from ucimlrepo import fetch_ucirepo

In [4]:
taiwanese_bankruptcy_prediction = fetch_ucirepo(id=572)
data = taiwanese_bankruptcy_prediction

In [5]:
X = data.data.features.to_numpy()
y = data.data.targets.to_numpy()
y = np.reshape(y, (6819,))

#### progress tracker

In [6]:
import sys
import time
def progress_bar(iteration, total, prefix='', suffix='', length=30, fill='█'):
    percent = ("{0:.1f}").format(100 * (iteration / float(total)))
    filled_length = int(length * iteration // total)
    bar = fill * filled_length + '-' * (length - filled_length)
    sys.stdout.write(f'\r{prefix} |{bar}| {percent}% {suffix}')
    sys.stdout.flush()

#### Calculating Mutual information

In [8]:
def compute_marginal_distances(feature, k):
    """
    Compute the distances to the k-th nearest neighbor for each point in a 1D feature.

    Parameters:
    feature : numpy array, shape (n_samples,)
        The 1D feature (column) in which to find nearest neighbors.
    k : int
        The number of nearest neighbors to consider.

    Returns:
    distances : numpy array, shape (n_samples,)
        The distance to the k-th nearest neighbor for each point.
    """
    n_samples = feature.shape[0]
    distances = np.zeros(n_samples)
    
    # Reshape the feature to 2D for KDTree (required by scipy.spatial.KDTree)
    feature_2d = feature.reshape(-1, 1)
    # start_time = time.time()
    # # Build a KDTree for efficient nearest neighbor search
    tree = KDTree(feature_2d)
    
    # Query the tree for the k-th nearest neighbor (excluding the point itself)
    distances, _ = tree.query(feature_2d, k=k+1)  # k+1 because the first neighbor is itself
    distances = distances[:, -1]  # Distance to the k-th neighbor
    # end_time = time.time()
    
    # print("elapsed_time", end_time - start_time)
    
    return distances

In [9]:
def count_points_within_distance(joint_data, marginal_distances_x, marginal_distances_y):
    """
    Count the number of points within the given marginal distances in the joint space.

    Parameters:
    joint_data : numpy array, shape (n_samples, 2)
        The joint data (X, Y).
    marginal_distances_x : numpy array, shape (n_samples,)
        The distances to the k-th nearest neighbor in the X-space.
    marginal_distances_y : numpy array, shape (n_samples,)
        The distances to the k-th nearest neighbor in the Y-space.

    Returns:
    n_x : numpy array, shape (n_samples,)
        The number of points within marginal_distances_x in the joint space.
    n_y : numpy array, shape (n_samples,)
        The number of points within marginal_distances_y in the joint space.
    """
    n_samples = joint_data.shape[0]
    n_x = np.zeros(n_samples)
    n_y = np.zeros(n_samples)

    # start_time = time.time()
    for i in range(n_samples):
        # Extract the query point (x_i, y_i)
        x_i, y_i = joint_data[i]
        
        # Compute the distance to all other points in the joint space
        distances_x = np.abs(joint_data[:, 0] - x_i)  # Distances in X-space
        distances_y = np.abs(joint_data[:, 1] - y_i)  # Distances in Y-space
        
        # Count points within epsilon_ix in X-space AND epsilon_iy in Y-space
        n_x[i] = np.sum((distances_x <= marginal_distances_x[i]) & (distances_y <= marginal_distances_y[i]))
        
        # Count points within epsilon_ix in X-space OR epsilon_iy in Y-space
        n_y[i] = np.sum((distances_x <= marginal_distances_x[i]) | (distances_y <= marginal_distances_y[i]))
    # end_time = time.time()
    
    # print("elapsed_time", end_time- start_time)
    
    return n_x, n_y

In [35]:
def ksg2_mutual_information(x, y, k = 10):
    """
    KSG2 estimator for mutual information between two continuous variables X and Y.

    Parameters:
    x : numpy array, shape (n_samples,)
        Samples from the marginal distribution of X.
    y : numpy array, shape (n_samples,)
        Samples from the marginal distribution of Y.
    k : int, optional
        The number of nearest neighbors to use (default is 4).

    Returns:
    mi : float
        The estimated mutual information.
    """
    n_samples = x.shape[0]
    
    # Stack X and Y to form the joint data
    joint_data = np.vstack((x, y)).T
    
    # Compute distances to the k-th nearest neighbor in the marginal spaces
    marginal_distances_x = compute_marginal_distances(x, k)
    marginal_distances_y = compute_marginal_distances(y, k)
    
    # Count points within these distances in the joint space
    n_x, n_y = count_points_within_distance(joint_data, marginal_distances_x, marginal_distances_y)
    
    # Compute mutual information using the KSG2 formula
    mi = digamma(k) - 1/k - np.mean(digamma(n_x) + digamma(n_y)) + digamma(n_samples)
    
    return mi

##### My Implementation of Mutual Information

In [37]:
Q_new = np.zeros((95, 95))
calculated = 0
total_calculation= 95 * (95)/2
for i in range(Q_new.shape[0]):
    for j in range(i, Q.shape[0]):
        Q_new[i,j] = ksg2_mutual_information(X[:,i], X[:,j])
        Q_new[j,i] = Q_new[i,j]
        calculated += 1
        progress_bar(calculated, total_calculation, prefix='Progress:', suffix='Complete', length=50)

Q_new[Q_new < 0] = 0
print(Q_new)

[[6.08887718 8.30817852 8.19101568 ... 8.09342996 0.         8.46844035]mplete
 [8.30817852 6.03554246 8.25532415 ... 8.05298415 0.         8.45435926]
 [8.19101568 8.25532415 6.04265425 ... 8.07659729 0.         8.45191729]
 ...
 [8.09342996 8.05298415 8.07659729 ... 5.57690639 0.         8.11749551]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [8.46844035 8.45435926 8.45191729 ... 8.11749551 0.         6.2756422 ]]


In [46]:
Q_new = np.round(Q_new, 1)
is_symmetric = np.array_equal(Q_new, Q_new.T)
print(is_symmetric)
df = pd.DataFrame(Q_new)
df

True


,0,1,2,3,4,5,6,7,8,9,...,85,86,87,88,89,90,91,92,93,94
0,6.1,8.3,8.2,8.4,8.4,8.4,8.4,8.4,8.3,8.4,...,8.4,8.5,8.5,8.5,8.4,8.5,8.1,8.1,0.0,8.5
1,8.3,6.0,8.3,8.4,8.4,8.4,8.4,8.3,8.3,8.4,...,8.2,8.5,8.4,8.4,8.3,8.5,8.1,8.1,0.0,8.5
2,8.2,8.3,6.0,8.4,8.4,8.4,8.4,8.4,8.3,8.4,...,8.4,8.5,8.5,8.4,8.4,8.5,8.1,8.1,0.0,8.5
3,8.4,8.4,8.4,6.1,6.7,8.4,8.4,8.4,8.4,8.4,...,8.5,8.5,8.5,6.2,8.5,8.5,8.1,8.1,0.0,8.5
4,8.4,8.4,8.4,6.7,6.1,8.4,8.4,8.4,8.4,8.4,...,8.5,8.5,8.5,6.8,8.5,8.5,8.1,8.1,0.0,8.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.4,8.5,...,8.5,8.5,8.5,8.5,8.5,6.3,8.1,8.1,0.0,6.3
91,8.1,8.1,8.1,8.1,8.1,8.1,8.1,8.1,8.0,8.1,...,8.1,8.2,8.1,8.1,8.1,8.1,5.6,5.6,0.0,8.1
92,8.1,8.1,8.1,8.1,8.1,8.1,8.1,8.1,8.0,8.1,...,8.1,8.2,8.1,8.1,8.1,8.1,5.6,5.6,0.0,8.1
93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
def diagonalize(matrix):
    """This function utlizes Eigendecomposistion to get 
    eigenvectors and eigenvalues, and it assumes the matrix is symmetric
    to approximate U^-1 == u ^ T
    """
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    U = np.diag(eigenvalues)
    return U, eigenvectors
eigenvalues, eigenvectors = diagonalize(Q_new)
print(eigenvalues)
print("")
print(eigenvectors)
print("")
check_Q = eigenvectors @ eigenvalues @ eigenvectors.T
print(check_Q)

[[6.66328845e-03 +0.j         0.00000000e+00 +0.j
  0.00000000e+00 +0.j         ... 0.00000000e+00 +0.j
  0.00000000e+00 +0.j         0.00000000e+00 +0.j        ]
 [0.00000000e+00 +0.j         1.77759596e+02 +0.j
  0.00000000e+00 +0.j         ... 0.00000000e+00 +0.j
  0.00000000e+00 +0.j         0.00000000e+00 +0.j        ]
 [0.00000000e+00 +0.j         0.00000000e+00 +0.j
  6.01691162e+01+85.78535705j ... 0.00000000e+00 +0.j
  0.00000000e+00 +0.j         0.00000000e+00 +0.j        ]
 ...
 [0.00000000e+00 +0.j         0.00000000e+00 +0.j
  0.00000000e+00 +0.j         ... 1.57485334e-01 +0.j
  0.00000000e+00 +0.j         0.00000000e+00 +0.j        ]
 [0.00000000e+00 +0.j         0.00000000e+00 +0.j
  0.00000000e+00 +0.j         ... 0.00000000e+00 +0.j
  3.63563973e-03 +0.j         0.00000000e+00 +0.j        ]
 [0.00000000e+00 +0.j         0.00000000e+00 +0.j
  0.00000000e+00 +0.j         ... 0.00000000e+00 +0.j
  0.00000000e+00 +0.j         4.62976804e-02 +0.j        ]]

[[ 0.00000000e+

##### sklearn Implementation of Mutual Information

In [7]:
from sklearn.feature_selection import mutual_info_regression
Q = np.zeros((95,95))
for j in range(Q.shape[1]):
    Q[j, j:] = mutual_info_regression(X[:, j:], X[:, j], discrete_features = False, n_neighbors = 10)
    Q[j:, j] = Q[j, j:]

np.save("taiwanese_dataset_Q", Q)
Q

array([[6.35303258, 1.30756376, 2.16620814, ..., 0.4435909 , 0.        ,
        0.0579472 ],
       [1.30756376, 6.33476042, 1.49799093, ..., 0.65448946, 0.        ,
        0.06570552],
       [2.16620814, 1.49799093, 6.33948949, ..., 0.44391168, 0.        ,
        0.06213331],
       ...,
       [0.4435909 , 0.65448946, 0.44391168, ..., 6.12716287, 0.        ,
        0.20580459],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.0579472 , 0.06570552, 0.06213331, ..., 0.20580459, 0.        ,
        6.4756422 ]], shape=(95, 95))

In [40]:
Q = np.round(Q, 1)
is_symmetric = np.array_equal(Q, Q.T)
print(is_symmetric)
df = pd.DataFrame(Q)
df

True


,0,1,2,3,4,5,6,7,8,9,...,85,86,87,88,89,90,91,92,93,94
0,6.4,1.3,2.2,0.2,0.2,0.5,0.7,0.6,0.1,0.7,...,1.2,0.0,0.0,0.2,1.0,0.1,0.3,0.4,0.0,0.1
1,1.3,6.3,1.5,0.2,0.2,0.5,0.9,0.9,0.2,0.8,...,2.3,0.0,0.0,0.2,1.7,0.1,0.6,0.7,0.0,0.1
2,2.2,1.5,6.3,0.2,0.2,0.5,0.7,0.6,0.1,0.8,...,1.3,0.0,0.0,0.2,1.0,0.1,0.3,0.4,0.0,0.1
3,0.2,0.2,0.2,6.4,5.3,0.5,0.3,0.3,0.0,0.3,...,0.2,0.0,0.0,6.4,0.2,0.1,0.1,0.2,0.0,0.1
4,0.2,0.2,0.2,5.3,6.4,0.5,0.3,0.3,0.0,0.3,...,0.2,0.0,0.0,5.3,0.2,0.1,0.1,0.2,0.0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.0,0.1,0.1,0.0,6.5,0.2,0.2,0.0,5.1
91,0.3,0.6,0.3,0.1,0.1,0.4,0.6,0.6,0.2,0.5,...,0.6,0.0,0.1,0.1,0.6,0.2,6.1,4.6,0.0,0.2
92,0.4,0.7,0.4,0.2,0.2,0.4,0.6,0.6,0.2,0.6,...,0.7,0.0,0.1,0.2,0.6,0.2,4.6,6.1,0.0,0.2
93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
from sklearn.feature_selection import mutual_info_classif
F = mutual_info_classif(X, y, n_neighbors = 7)
np.save("taiwanese_dataset_F", F)

####                                                        Nystrom Approximation

##### Diagonalization of Redundency

In [54]:
def diagonalize(matrix):
    """This function utlizes Eigendecomposistion to get 
    eigenvectors and eigenvalues, and it assumes the matrix is symmetric
    to approximate U^-1 == u ^ T
    """
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    U = np.diag(eigenvalues)
    return U, eigenvectors
eigenvalues, eigenvectors = diagonalize(Q)
print(pd.DataFrame(np.round(eigenvalues, 1)))
print(" ")
print(eigenvectors)
print(" ")
check_Q = eigenvectors @ eigenvalues @ eigenvectors.T
print(check_Q)

      0     1     2     3     4    5    6    7    8    9   ...   85   86   87  \
0   26.6   0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1    0.0  24.2   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2    0.0   0.0  17.8   0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3    0.0   0.0   0.0  17.1   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4    0.0   0.0   0.0   0.0  13.5  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
..   ...   ...   ...   ...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
90   0.0   0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
91   0.0   0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
92   0.0   0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
93   0.0   0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
94   0.0   0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

     88   89   90   91   92

##### Estimation of rank for Nystrom approximation

In [56]:
from scipy.sparse.linalg import eigsh
def approx_rank_lanczos(A, tol=1e-10, k=100):
    """Estimate rank using the Lanczos algorithm with k eigenvalues."""
    eigenvalues = eigsh(A, k=k, return_eigenvectors=False)
    return np.sum(np.abs(eigenvalues) > tol)
# Example: Large symmetric matrix
rank = approx_rank_lanczos(Q_new)
print("Approximate Rank (Lanczos):", rank)

Approximate Rank (Lanczos): 86


C:\Users\amirf\AppData\Local\Temp\ipykernel_3264\1830813076.py:4: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues = eigsh(A, k=k, return_eigenvectors=False)


##### Uniform sampling without replecment to extract A and B

In [ ]:
row_indices = np.random.choice(Q_new.shape[0], size = rank, replace = False)
random_rows = Q_new[row_indices]
# Now take those rows and divide them into A and B
A = random_rows[:,:rank]
B = random_rows[:, rank:]
print(random_rows)
print("A", A)
print("B", B)

##### Determining the intermidate matrix and Diagonalization

In [22]:
from scipy.linalg import sqrtm, inv

##### calculating inverse square matrix

In [ ]:
def inv_sqrt(A):
    if not is_psd(A):
      raise ValueError("Matrix A is not positive semi-definite.")
    A_diag = diagonalize(A)
    E = inv(sqrtm(A_diag[0]))
    A_inv_sqrm = A_diag[1] @ E @ A_diag[1].T
    return A_inv_sqrm

##### Calculating intermidate matrix

In [ ]:
def intermidiate_matrix(A, B):
    A_inv_sqrm = inv_sqrt(A)
    S = A + A_inv_sqrm @ B @ B.T @ A_inv_sqrm
    return S

##### Calculating second_diagonalization

In [ ]:
def compute_eigenvectors(A, B):
    S = intermidiate_matrix(A, B)
    S_diag = diagonalize(S)
    eigenvalues = S_diag[0]
    R = S_diag[1]
    first_val = np.concatenate((A, B.T), axis = 0)
    second_val = inv_sqrt(A)
    fourth_val = inv_sqrt(eigenvalues)
    eigenvectors = first_val @ second_val @ R @ fourth_val
    return eigenvalues, eigenvectors

##### Calculating alpha

In [ ]:
q = np.mean(random_rows)
f = np.mean(MR)
alpha = q/(q+f)

In [20]:
sol = np.load("solution.npy")
sol[sol != 0]

array([1.000000e-07, 1.204000e-04, 5.340000e-04, 7.000000e-07,
       5.730000e-05, 2.169000e-04, 3.850000e-05, 1.637000e-04,
       9.036000e-04, 1.900000e-06, 1.000000e-07, 5.864629e-01,
       1.297000e-04, 4.113702e-01])

array([[0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 1.000000e-07, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 1.204000e-04, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 5.340000e-04, 7.000000e-07, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 5.730000e-05, 0.000000e+00,
        2.169000e-04, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 3.850000e-05, 1.637000e-04, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 9.03